In [8]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

model_name="BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cpu","trust_remote_code":True}
encode_kwargs = {"normalize_embeddings": True}


# text_splitter = RecursiveCharacterTextSplitter(chunk_size=250,
    #                                                    chunk_overlap=50)




embedding_function = SentenceTransformerEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs,
)



/home/hp/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
file_path = 'contract.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()


In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import time     
def get_docs(text):
    print("Executing chunker ....")
    docs = Document(page_content=text, metadata={"source": "book"})
    doc = [docs]
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_overlap=0,
        chunk_size=1200,
        separators=["*****", "\n\n", "\n", "\uff0e", "\u3002"]
    )
    return text_splitter.split_documents(doc)

def initialize_dbs_for_documents(text, path):
    if not os.path.exists(path):
        print("if")
        db = Chroma.from_documents(get_docs(text), embedding_function, persist_directory=path)
        time.sleep(8)
        vectordb = Chroma(persist_directory=path, embedding_function=embedding_function)
        return vectordb
    else:
        print("else")
        vectordb = Chroma(persist_directory=path, embedding_function=embedding_function)
        return vectordb
    

vectorbb=initialize_dbs_for_documents(text,path="embed")

In [9]:
vectordb = Chroma(persist_directory="embed",embedding_function=embedding_function)
vectordb

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def check_token_limit(new_documents):
  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
  tokens=tokenizer(new_documents)
  n_tokens=len(tokens.input_ids)
  return n_tokens

In [11]:
k=100
def search(prompt):
  print("-----------")
  print(prompt)
  docs = vectordb.similarity_search_with_score(query=prompt,k=k)
  # print("Documents:")
  # print(len(docs))
  # print("::Documents")
  new_documents=" "
  for i in range(0,len(docs)):
    temp_doc=docs[i][0].page_content
    # print("Chunk : : \n : "+ temp_doc)
    check_doc=new_documents+temp_doc
    tokens=check_token_limit(check_doc)
    if tokens>4000:
      doc=temp_doc
      for x in range(0,5):
        doc= doc[:len(doc) // 2]
        temp_tokens=check_token_limit(doc+new_documents)
        if temp_tokens<=3950:
          new_documents =new_documents+doc + "\n\n--------------------\n\n"
          break


      return new_documents
    else:
      new_documents=check_doc + "\n\n-----------------\n\n"

  return new_documents

In [5]:
system_prompt="""You are a knowledgeable and professional assistant tasked with providing detailed information about construction contract. Your responses should be concise, informative, and strictly adhere to the terms and conditions outlined in the provided contract details."""

In [6]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage


# llm = Ollama(model="llama3:8b-text-q4_K_M")
 
llm = ChatOllama(
    model="llama3:8b-instruct-q4_K_M",
    n_ctx=8000,
    temperature=0.0,
    top_p=0.7,
    
    system_message=SystemMessage(content=system_prompt)
)


In [13]:
prompts = [
    "What is CONTRACTOR limit of liability and the liability cap?",
    "Under what condition COMPANY can terminate the contract?",
    "Can CONTRACTOR terminate the contract?",
    "Can CONTRACTOR claim extension of time and additional cost on account of force majeure?",
    "Is increase in quantity a COMPANY risk event or CONTRACTOR risk event?",
]


In [ ]:
new_documents=search(prompt[0])
question = " <Data>: \n" + new_documents + "\n\n ------------------- "+" \n\n <Question>: "+prompt[0]
llm.invoke(question)

In [21]:
new_documents=search(prompt[0])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[0]
llm.invoke(question)


-----------
What is CONTRACTOR limit of liability and the liability cap?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

AIMessage(content='Based on the provided text, there is no specific mention of a "CONTRACTOR limit of liability" or a "liability cap". However, it can be inferred that the CONTRACTOR\'s liability is limited in certain situations:\n\n1. Article 33.2(a) states that the CONTRACTOR shall rectify any loss or damage to the WORKS prior to issuance of the PROVISIONAL ACCEPTANCE CERTIFICATE at its sole cost and expense.\n2. Article 42.4 Pollution from CONTRACTOR PROPERTY mentions that the CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, claims, costs, liabilities, damages, and expenses arising from or in connection with any pollution originating from the CONTRACTOR\'s performance of the WORKS and/or the AGREEMENT.\n3. Article 43.1 states that the CONTRACTOR shall not assign, novate, or otherwise transfer any of its rights, obligations, or interests under and in connection with the AGREEMENT without the prior written consent of the COMPANY

In [ ]:
new_documents=search(prompt[2])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[2]
llm.invoke(question)  

In [ ]:
new_documents=search(prompt[3])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[3]
llm.invoke(question)  

In [ ]:
new_documents=search(prompt[4])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[4]
llm.invoke(question)  

### Ollama using local tunned

In [ ]:
import requests
import json
for prompt in prompts:
 
 new_documents=search(prompt)
 question = " <Data>: \n" + new_documents + "\n\n ------------------- "+" \n\n <Question>: "+prompt
 
 url = "http://localhost:11434/api/generate"

 payload = {
    "model": "llama3:8b-instruct-q4_K_M",
    "options": {"num_ctx": 8000,"temperature":0.0,"top_p":0.5},
    "system":system_prompt,
    "prompt": question,
    "stream": False
 }

# Send the POST request
 response = requests.post(url, data=json.dumps(payload), headers={"Content-Type": "application/json"})

# Print the response

 if response.status_code == 200:
    print("Response:", response.json())
 else:
    print(f"Failed to get a response. Status code: {response.status_code}, Response: {response.text}")


top p=0.7

According to Article 41.5(a) of the agreement, the CONTRACTOR's limit of liability for Third Party Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10, the total liability of either PARTY (i.e., the COMPANY or the CONTRACTOR) to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this text.\n\nHowever, it can be inferred from Article 42.11(a) that the maximum amount of consequential loss that the COMPANY INDEMNIFIED PARTIES may recover from the CONTRACTOR INDEMNIFIED PARTIES is USD 1,000,000 per occurrence

default 

According to Article 41.5, the CONTRACTOR's Third Party Liability Insurance has a limit of not less than USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT.\n\nThe total liability cap is 100% of the AGREEMENT PRICE."

temperature=0   

According to Article 41.5, the CONTRACTOR's limit of liability for Third Party Liability Insurance and Pollution Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this text.

temperature =0 top_p=0.7

Based on the provided text, it appears that there are several instances where the CONTRACTOR's liability is limited or capped. Here are some specific examples:\n\n1. Article 42.3: The CONTRACTOR shall indemnify the COMPANY INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES.\n2. Article 42.12 (ii): The COMPANY shall indemnify the CONTRACTOR INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the CONTRACTOR INDEMNIFIED PARTIES.\n\nIt appears that the liability cap is USD 1,000,000 per occurrence. However, it's worth noting that there may be other instances where the liability is limited or capped, and these examples are not exhaustive.", 


temperature = 0  top_p=0.5

"According to Article 41.5, the CONTRACTOR's limit of liability for Third Party Liability Insurance and Pollution Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this document. However, it can be inferred that the total liability cap is 100% of the AGREEMENT PRICE, as mentioned in Article 42.10(a)